In [3]:
def elves():
    with open("day1.txt") as f:
        data=[line.strip() for line in f.readlines()]
        def chunks():
            p = 0
            for d in data:
                if d=='':
                    yield p
                    p = 0
                else:
                    p += int(d)
        return chunks()
print(max(elves()))

68923


In [8]:
print(sum(sorted(elves(), reverse=True)[0:3]))

200044
